In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 필터 커널 세팅
kernel = np.ones((1, 1), np.uint8)
kernel2 = np.ones((3, 3), np.uint8)

In [6]:
############################################################################################################
    # 마지막 코드 #
############################################################################################################

# 영상 경로 설정
video_path= './video2.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

while True :
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
        
    # img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))    
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img2 = yuv[:, :, 2]

    thr, mask_red = cv2.threshold(img2, 140, 255, cv2.THRESH_BINARY)
    
    # 침식 (노이즈제거) -> 팽창(픽셀 살리기)
    erosion_mask = cv2.erode(mask_red, kernel, iterations=1)
    dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 
    cv2.imshow('dilation_mask', dilation_mask)
    contours, hierarchy = cv2.findContours(dilation_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    mask_external = np.zeros(dilation_mask.shape, dilation_mask.dtype)
    for i in range(len(contours)) :
        if hierarchy[0][i][3] == -1 :
            cv2.drawContours(mask_external, contours, i, 255, -1)

    mask_internal = np.zeros(dilation_mask.shape, dilation_mask.dtype)
    for i in range(len(contours)) :
        if hierarchy[0][i][3] != -1 :
            cv2.drawContours(mask_internal, contours, i, 255, -1)
                    
    _, _, stats_i, centroids_i = cv2.connectedComponentsWithStats(mask_internal)
    _, _, stats_e, centroids_e = cv2.connectedComponentsWithStats(mask_external)
    
    for idx, centroid_i in enumerate(centroids_i) :
        
        if stats_i[idx][0] == 0 and stats_i[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid_i)) :
            continue
            
        for centroid_e in centroids_e :
            if abs(centroid_i[0] - centroid_e[0]) < 3 and abs(centroid_i[1] - centroid_e[1]) < 3 :
                x, y, w, h, area = stats_i[idx]
                centerX, centerY = int(centroid_i[0]), int(centroid_i[1])

                if area > 120 and abs(w-h) < 5 :
                    try : 
                        detected_img = img[y-10 : y+h+10,  x-10 : x+w+10]
                        gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                        circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=30)
                        if circles is not None :
                            circles = np.uint16(np.around(circles))

                            for i in circles[0, :] :
                                cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)
                                cv2.rectangle(img, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
                    except : 
                        continue

    cv2.imshow('img', img)
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 필터 커널 세팅
kernel = np.ones((1, 1), np.uint8)
kernel2 = np.ones((3, 3), np.uint8)

############################################################################################################
    # 마지막 코드 #
############################################################################################################

# 영상 경로 설정
video_path= './video3.mp4'
cap = cv2.VideoCapture(video_path)

# 영상 이름
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 필터 
red1 = np.array([0, 10, 10]) # 밝은 빨강(주황계열)
red2 = np.array([7, 250, 240])

red3 = np.array([155, 10, 0]) # 어두운 빨강(갈색계열)
red4 = np.array([180, 250, 240])

while True :
    # 영상 읽기
    ret, img = cap.read()
    
    # 마지막 프레임 확인
    if ret is None:
        break
    
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    mask_red = mask_red1 + mask_red2    
    # cv2.imshow('mask_red', mask_red)
    
    res_red = cv2.bitwise_and(img, img, mask = mask_red)
    # cv2.imshow('res_red', res_red )
    
    # img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2))    
    thr, mask_red = cv2.threshold(res_red, 140, 255, cv2.THRESH_BINARY)
    
    res_gray = cv2.cvtColor(res_red, cv2.COLOR_BGR2GRAY)
    thr, mask_gray = cv2.threshold(res_gray, 60, 255, cv2.THRESH_BINARY)
    
    # 침식 (노이즈제거) -> 팽창(픽셀 살리기)
    erosion_mask = cv2.erode(mask_gray, kernel, iterations=1)
    dilation_mask = cv2.dilate(erosion_mask, kernel2, iterations=1) 
    cv2.imshow('dilation_mask', dilation_mask)
    contours, hierarchy = cv2.findContours(dilation_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    mask_external = np.zeros(dilation_mask.shape, dilation_mask.dtype)
    for i in range(len(contours)) :
        if hierarchy[0][i][3] == -1 :
            cv2.drawContours(mask_external, contours, i, 255, -1)

    mask_internal = np.zeros(dilation_mask.shape, dilation_mask.dtype)
    for i in range(len(contours)) :
        if hierarchy[0][i][3] != -1 :
            cv2.drawContours(mask_internal, contours, i, 255, -1)
                    
    _, _, stats_i, centroids_i = cv2.connectedComponentsWithStats(mask_internal)
    _, _, stats_e, centroids_e = cv2.connectedComponentsWithStats(mask_external)
    
    for idx, centroid_i in enumerate(centroids_i) :
        
        if stats_i[idx][0] == 0 and stats_i[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid_i)) :
            continue
            
        for centroid_e in centroids_e :
            if abs(centroid_i[0] - centroid_e[0]) < 3 and abs(centroid_i[1] - centroid_e[1]) < 3 :
                x, y, w, h, area = stats_i[idx]
                centerX, centerY = int(centroid_i[0]), int(centroid_i[1])

                if area > 120 and abs(w-h) < 5 :
                    try : 
                        detected_img = img[y-10 : y+h+10,  x-10 : x+w+10]
                        gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                        circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=100, param2=30)
                        if circles is not None :
                            circles = np.uint16(np.around(circles))

                            for i in circles[0, :] :
                                cv2.circle(img, ( centerX, centerY ), 2, (0, 255, 0), 2)
                                cv2.rectangle(img, (x-10, y-10), (x+w+10, y+h+10), (0, 0, 255), 2)
                    except : 
                        continue

    cv2.imshow('img', img)
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.1.1) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
